## Constants and Imports

In [ ]:
from functions_analysis import *
from functions_filtering import *
import glob
import pandas as pd
from DockQ.DockQ import load_PDB, run_on_all_native_interfaces
from download_functions import *
import os
import itertools

In [ ]:
review_files = glob.glob('/home/markus/MPI_local/production1/structure_reviews/*.csv')
# review_files = glob.glob('/home/markus/MPI_local/production1/structure_reviews/intersect_df - set3.csv')
reviews_df = pd.concat([pd.read_csv(f) for f in review_files], ignore_index=True)
reviews_df = reviews_df.drop_duplicates(subset=['pdb_id', 'query_tf', 'query_arm', 'chain_tf', 'chain_arm'])

## Download PDBs

In [ ]:
good_structs = reviews_df[reviews_df['comment'].str.contains('looks good|in complex', case=False, na=False)]

# Get unique PDB IDs from reviews_df
unique_pdb_ids = good_structs['pdb_id'].unique()
print(f"Found {len(unique_pdb_ids)} unique PDB IDs to download")

# Define download directory
pdb_download_dir = "/home/markus/MPI_local/data/PDB"

# Download each PDB structure (function handles duplicate checking)
downloaded_count = 0
failed_count = 0

for pdb_id in unique_pdb_ids:
    if pd.isna(pdb_id):  # Skip NaN values
        continue
        
    result = download_pdb_structure(pdb_id, pdb_download_dir)
    if result:
        downloaded_count += 1
    else:
        failed_count += 1

print(f"\nSummary:")
print(f"Successfully processed: {downloaded_count}")
print(f"Failed downloads: {failed_count}")
print(f"Total processed: {len([pdb_id for pdb_id in unique_pdb_ids if not pd.isna(pdb_id)])}")

In [ ]:
# test get_all_chain_mappings()
# Case 1: model_chains and native_chains have same length
native_chains = ['A', 'B']
model_chains = ['X', 'Y']
mappings = list(get_all_chain_mappings(native_chains, model_chains))
assert mappings == [{'A': 'X', 'B': 'Y'}, {'A': 'Y', 'B': 'X'}]
# Case 2: model_chains > native_chains
native_chains = ['A', 'B', 'C']
model_chains = ['X', 'Y']
mappings = get_all_chain_mappings(native_chains, model_chains)
assert mappings == [{'A': 'X', 'B': 'Y'}, {'A': 'Y', 'B': 'X'}, {'A': 'X', 'C': 'Y'}, {'A': 'Y', 'C': 'X'}, {'B': 'X', 'C': 'Y'}, {'B': 'Y', 'C': 'X'}]

## calculate DockQ

In [ ]:
PROTEOME_PATHS = [
    '/home/markus/MPI_local/data/Proteome/uniprotkb_proteome_UP000005640_2025_05_28.tsv',
    '/home/markus/MPI_local/data/full_UP/uniprotkb_AND_reviewed_true_2025_07_10.tsv'
]

all_uniprot = pd.concat(
    [pd.read_csv(path, low_memory=False, sep='\t') for path in PROTEOME_PATHS],
    ignore_index=True
)

In [ ]:
NATIVE_PATH_PREFIX = "/home/markus/MPI_local/data/PDB/"
HPC_FULL_RESULTS_DIR = "/home/markus/MPI_local/HPC_results_full"

# Calculate DockQ scores using the function from functions_filtering.py
good_structs = calculate_dockq_scores(good_structs, NATIVE_PATH_PREFIX, HPC_FULL_RESULTS_DIR, all_uniprot)

In [ ]:
# set pair_id
good_structs['pair_id'] = good_structs.apply(lambda row: str(tuple(sorted([row['query_arm'].split('|')[0].upper(), row['query_tf'].split('|')[0].upper()]))), axis=1)

In [ ]:
RESULTS_DIR_ALL = "/home/markus/MPI_local/HPC_results"
from functions_analysis import *

In [ ]:
results_df_all_uc = pd.DataFrame(data=find_summary_files(RESULTS_DIR_ALL))
print(f"Total jobs processed: {len(results_df_all_uc)}")
results_df_all_uc['pair_id'] = results_df_all_uc.apply(create_pair_id, axis=1)
results_df_all = clean_results(results_df_all_uc)

In [ ]:
good_structs_annotated = pd.merge(good_structs, results_df_all, how='left', on='pair_id')

In [ ]:
from functions_plotting import *

In [ ]:
create_scatter_plot(good_structs_annotated, 'ranking_score', 'dockq_score')
create_scatter_plot(good_structs_annotated, 'iptm', 'dockq_score')
create_scatter_plot(good_structs_annotated, 'ptm', 'dockq_score')

In [ ]:
from Bio.PDB import MMCIFParser, PDBIO

In [ ]:
def get_pdb_path(cif_path: str, pdb_dir: str) -> str:
    """for the .cif structure at cif_path, check if there is a pdb file available in pdb_dir.
    If not, convert the .cif to a pdb file and store it in pdb_dir. Return the path to the pdb file

    Args:
        cif_path (str): Path to the input CIF file
        pdb_dir (str): Directory to store/find PDB files

    Returns:
        str: Path to the PDB file
    """
    # Create pdb_dir if it doesn't exist
    os.makedirs(pdb_dir, exist_ok=True)
    
    # Extract filename without extension from cif_path
    cif_filename = os.path.basename(cif_path)
    pdb_filename = os.path.splitext(cif_filename)[0] + '.pdb'
    pdb_path = os.path.join(pdb_dir, pdb_filename)
    
    # Check if PDB file already exists
    if os.path.exists(pdb_path):
        return pdb_path
    
    # Convert CIF to PDB
    parser = MMCIFParser(QUIET=False)
    structure = parser.get_structure("structure", cif_path)
    
    # Write to PDB
    io = PDBIO()
    io.set_structure(structure)
    io.save(pdb_path)
    
    return pdb_path

In [ ]:
import subprocess
from pathlib import Path

TM_BIN = '/home/markus/MPI_local/bin/TMscore'
USALIGN_BIN = '/home/markus/MPI_local/bin/USalign'
def calc_USalign(model, native, output_dir):
    os.makedirs(output_dir, exist_ok=True)
    model_name = Path(model).stem
    native_name = Path(native).stem
    
    out_path = f"{output_dir}/{model_name}--{native_name}"
    
    if os.path.exists(out_path):
        return out_path
    
    try:
        with open(out_path, 'w') as output_file:
            subprocess.run([
                USALIGN_BIN,
                model,
                native,
                "-ter", "1",
                "-mm", "1",
                "-outfmt", "2"
            ], stdout=output_file, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")
        
    return out_path

In [ ]:
PDB_DIR = '../../production1/pdb_cache'
ALIGN_DIR = '../../production1/us_align_out'


us_align_score_df = pd.DataFrame()
error_count = 0
no_model = []
no_native = []

for index, row in good_structs.iterrows():
    
    job_name = get_job_name(row['query_tf'].split('|')[0], row['query_arm'].split('|')[0], all_uniprot)
    model_path_cif = get_file_path(f'{job_name}_model.cif', HPC_FULL_RESULTS_DIR)
    native_path_cif = f'{NATIVE_PATH_PREFIX}{row["pdb_id"].lower()}.cif'
    
    
    # Check if both paths exist before loading
    if not model_path_cif:
        no_model.append((model_path_cif,job_name))
        continue

    if not os.path.exists(native_path_cif):
        no_native.append((native_path_cif,job_name))
        continue
    
    try:
        native_path_pdb = get_pdb_path(native_path_cif, PDB_DIR)
        model_path_pdb = get_pdb_path(model_path_cif, PDB_DIR)
        us_align = calc_USalign(model_path_pdb, native_path_pdb, ALIGN_DIR)
        us_align_df = pd.read_csv(us_align, sep='\t')
        us_align_df['job_name'] = job_name
        us_align_df['pdb_id'] = row['pdb_id']
        us_align_score_df = pd.concat([us_align_score_df, us_align_df], ignore_index=True)
    except Exception as e:
        error_count += 1
        print(e)
        
print(error_count)
print(f"{len(no_model)}")
print(f"{len(no_native)}")

for list in no_model:
    print(list)
    